# Experiment Parameters

In [1]:
dino_dataset_names = ["headlines-naive-7600-dataset.jsonl", "headlines-processed-human-dataset.jsonl"]
model_name = "distilbert-base-uncased" # bert-base-uncased
num_examples = 6200
shrink_eval = -1
batch_sz = 32
num_epochs = 2
num_evals_per_epoch = 3
learning_rate = 5e-5

# Installs

In [2]:
!pip install transformers datasets evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 5.0 MB/s eta 0:00:00


# Git

In [3]:
!git clone https://github.com/jasony123123/dino

Cloning into 'dino'...
remote: Enumerating objects: 155, done.
remote: Counting objects: 100% (46/46), done.
remote: Compressing objects: 100% (34/34), done.
remote: Total 155 (delta 22), reused 24 (delta 9), pack-reused 109
Receiving objects: 100% (155/155), 2.84 MiB | 6.34 MiB/s, done.
Resolving deltas: 100% (68/68), done.


In [4]:
%cd dino/datasets

/kaggle/working/dino/datasets


# Load Datasets

In [5]:
from datasets import load_dataset

real_dataset = load_dataset("ag_news")

Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7600 [00:00<?, ? examples/s]

Dataset ag_news downloaded and prepared to /root/.cache/huggingface/datasets/ag_news/default/0.0.0/bc2bcb40336ace1a0374767fc29bb0296cdaf8a6da7298436239c54d79180548. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [6]:
import json
import datasets

def read_jsonl_dataset(filename):
  train_dataset = datasets.load_dataset("json", data_files=filename)
  labels = {"world": 0, "sports": 1, "business": 2, "scitech": 3}
  mapping = lambda example: {"text": example["text_a"], "label": labels[example["label"]]}

  train_dataset = train_dataset.map(mapping, remove_columns=["text_a","text_b"])
  final_dataset = datasets.DatasetDict({"train": train_dataset["train"]})
  return final_dataset

dino_datasets = [(read_jsonl_dataset(fname), fname) for fname in dino_dataset_names]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-792d722a06794d92/0.0.0/ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/7637 [00:00<?, ?ex/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-e43263dc3b4a1967/0.0.0/ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/6235 [00:00<?, ?ex/s]

In [7]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_name)
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_real_datasets = real_dataset.map(tokenize_function, batched=True)
tokenized_dino_datasets = [
    (data.map(tokenize_function, batched=True), name)
    for data, name in dino_datasets
]

  0%|          | 0/120 [00:00<?, ?ba/s]

  0%|          | 0/8 [00:00<?, ?ba/s]

  0%|          | 0/8 [00:00<?, ?ba/s]

  0%|          | 0/7 [00:00<?, ?ba/s]

# Experiment

In [8]:
small_eval_real_dataset = tokenized_real_datasets["test"].shuffle(seed=42)
if shrink_eval > 0:
  small_eval_real_dataset = small_eval_real_dataset.select(range(shrink_eval))

null_real_dataset = tokenized_real_datasets["train"].shuffle(seed=42).select(range(1))
small_train_real_dataset = tokenized_real_datasets["train"].shuffle(seed=42).select(range(num_examples))
small_train_dino_datasets = [
    (data["train"].shuffle(seed=42).select(range(num_examples)), name)
    for data, name in tokenized_dino_datasets
]

In [9]:
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments
import numpy as np
import evaluate
from transformers import TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=4)

eval_steps = (num_examples / batch_sz) // num_evals_per_epoch
training_args = TrainingArguments(
    output_dir="test_trainer",
    evaluation_strategy="steps",
    num_train_epochs=num_epochs,
    per_device_train_batch_size=batch_sz,
    per_device_eval_batch_size=batch_sz,
    learning_rate=learning_rate,
    eval_steps=eval_steps,
    logging_strategy="steps",
    logging_steps=eval_steps,
    report_to="none",
)
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_projector.bias', 'vocab_projector.weight', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'classifier.weight', 'classifier

In [10]:
training_datasets = [
    # (null_real_dataset, "none"),
    (small_train_real_dataset, "real"),
    *small_train_dino_datasets
]

for train_dataset, name in training_datasets:
  trainer = Trainer(
      model=model,
      args=training_args,
      train_dataset=train_dataset,
      eval_dataset=small_eval_real_dataset,
      compute_metrics=compute_metrics,
  )

  print("------------------")
  print(name)
  print("------------------")
  trainer.train()

------------------
real
------------------


/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss,Accuracy
64,0.587500,0.348796,0.887895
128,0.344500,0.312771,0.898026
192,0.308500,0.279574,0.908816
256,0.222200,0.281919,0.908947
320,0.203700,0.283824,0.907237
384,0.188900,0.272616,0.911579


------------------
headlines-naive-7600-dataset.jsonl
------------------


/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss,Accuracy
64,0.033000,1.340805,0.727763
128,0.007200,1.360476,0.748684
192,0.003000,1.232789,0.775789
256,0.000400,1.522799,0.746184
320,0.000300,1.536974,0.746842
384,0.000300,1.528127,0.749737


------------------
headlines-processed-human-dataset.jsonl
------------------


/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss,Accuracy
64,0.047500,0.832493,0.866184
128,0.005500,0.845244,0.867763
192,0.002900,0.823596,0.873684
256,0.000200,0.834539,0.876974
320,0.000100,0.844229,0.877237
384,0.000100,0.846792,0.877500


<a id='pytorch_native'></a>

<a id='additional-resources'></a>